In [1]:
import sys
import os
import random
sys.path.insert(0, '../../dev')

In [2]:
import os
import numpy as np
import pandas as pd
from song_classification_v2 import *

# musicautobot
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.utils.midifile import *
from musicautobot.utils.file_processing import process_all

import music21 as m21
from music21 import *

In [4]:
df = pd.read_csv("./songname_to_msid.txt", delimiter="   ", header=None, engine='python')

df.columns = ['Artist', 'Song', 'Id']
print(df.head())
print('----------------------')
for row in df.iterrows():
    print(row[1]['Id'])
    break

print('----------------------')
print(df.loc[df['Id'] == 'TRBOEFO128F92FC62E'])

                                              Artist  \
0  Marilyn Horne / The New Symphony Orchestra Of ...   
1                                      Irwin Goodman   
2                                    Louis Bertignac   
3                          The Unborn Chikken Voices   
4                                          Aerosmith   

                              Song                  Id  
0          Hence_ Iris_ hence away  TRRNARX128F4264AEB  
1                   Sinua rakastan  TRWMHMP128EF34293F  
2  (Sittin'On) The Dock Of The Bay  TRWOLRE128F427D710  
3                  Again And Again  TRAPFCP128F92D772E  
4                         Dream On  TRBOEFO128F92FC62E  
----------------------
TRRNARX128F4264AEB
----------------------
      Artist      Song                  Id
4  Aerosmith  Dream On  TRBOEFO128F92FC62E


In [9]:
mf = music21.midi.MidiFile()
np_path = "C:\\Users\\bkweb\\Documents\\dragn\\MusGen\\src\\numpy_data_collections\\numpy_path_0_to_200\\TRBOEFO128F92FC62E.npy"
mid_path = "C:\\Users\\bkweb\\Documents\\dragn\\MusGen\\src\\mid_data_collections\\mid_0_to_200\\TRBOEFO128F92FC62E.mid"
mf.open(mid_path)
mf.read()
mf.close()
ss = music21.midi.translate.midiFileToStream(mf)
npy = np.load(np_path)
musIt = MusicItem.from_npenc(npy, MusicVocab.create())
calculate_score(musIt)
# ss.show()

('Ab_major',
 Counter({'G#|Ab': 250,
          'F': 418,
          'C': 403,
          'G': 248,
          'D#|Eb': 189,
          'D': 58,
          'C#|Db': 160,
          'A#|Bb': 130,
          'E': 25,
          'A': 3,
          'F#|Gb': 3,
          'B': 8}))

In [64]:
path = "C:\\Users\\bkweb\\Documents\\dragn\\MusGen\\src\\mid_data_collections\\mid_0_to_200\\TRBOEFO128F92FC62E.mid"
m = music21.midi.MidiFile()
m.open(path)
m.read()

tracks = [t for t in m.tracks if not any([e.channel == 10 for e in t.events])]        
score = music21.stream.Score()
music21.midi.translate.midiTracksToStreams(tracks,                                            
    inputM21=score,
    forceSource=False, 
    quantizePost=False,
    ticksPerQuarter=m.ticksPerQuarterNote,
    quarterLengthDivisors=(4,3),
)       
# score.write('midi', fp='out.midi')
classification = score.analyze('key')

In [100]:
folders_path = '../numpy_data_collections'
folders = os.listdir(folders_path)
print('folders: ', folders)

count = 0

everything = []
collection = []
new_df = pd.DataFrame(collection, columns=['Artist', 'Song', 'Id', 'Aard_key', 'Aard_r', 'KS_key', 'KS_r', 'Trevor_key'])
new_df.to_csv('classified_songs_V3.csv', sep='\t', index=False, header=True)

mf = music21.midi.MidiFile()
score = music21.stream.Score()
vocab = MusicVocab.create()
for sub_folder in folders:
    files = os.listdir(folders_path + '/' + sub_folder)
    if sub_folder == 'numpy_0_to_10000':
        continue
    print(sub_folder)
    for file in files:
        count += 1
        if count > 5:
            break
        filepath = folders_path + '/' + sub_folder + '/' + file
        print(f"filepath: {filepath}")
        
        # Load numpy tensor from file
        npy = np.load(filepath)
        # Create music item from numpy tensor
        musIt = MusicItem.from_npenc(npy, MusicVocab.create())
        # Generate m21 score from music item
        mf = m21.midi.translate.streamToMidiFile(musIt.to_stream())

        # identify the non-percussion tracks
        tracks = [t for t in mf.tracks if not any([e.channel == 10 for e in t.events])]  
        # translate the non-percussion tracks to a m21 stream
        score = music21.midi.translate.midiTracksToStreams(tracks,                                            
            # inputM21=score, # I don't think we actually want this line in here...
            forceSource=False, 
            quantizePost=False,
            ticksPerQuarter=mf.ticksPerQuarterNote,
            quarterLengthDivisors=(4,3),
        )
        
        # Analyze the key of the m21 stream with the Aarden-Essen algorithm (their default)
        aardenKey = score.analyze('Aarden')
        aard_tonic = aardenKey.tonic.name
        aard_mode = aardenKey.mode
        aard_r = aardenKey.correlationCoefficient

        # Analyze the key of the m21 stream with the Krumhansl-Schmuckler algorithm
        ksKey = score.analyze('Krumhansl')
        ks_tonic = ksKey.tonic.name
        ks_mode = ksKey.mode
        ks_r = ksKey.correlationCoefficient

        # Analyze the key of the MusicItem with Trevor's algorithm
        trevKey, occ_scores = calculate_score(musIt)
        # Reformat Trevor's keys to match m21's keys
        trevKey = trevKey.replace('_', ' ')
        trevKey = trevKey.replace('b', '-')

        print(f"\tAarden\t\tclassification: {aard_tonic} {aard_mode}\t r = {aard_r}")
        print(f"\tKrumhansl\tclassification: {ks_tonic} {ks_mode}\t r = {ks_r}")
        print(f"\tTrevor\t\tclassification: {trevKey}")


        # get artist and song
        row = df.loc[df['Id'] == file[:-4]]
        #if row != None:
        if file[:-4] in df.values:
            artist = row['Artist'].values[0]
            song = row['Song'].values[0]
            id = row['Id'].values[0]

        else:
            artist = 'N/A'
            song = 'N/A'
            id = row['Id']

        # print(artist, ', ', song, ', ', id, ', ', classification)
        # insert row into collection (artist, song, id)
        collection.append([artist, song, id, aard_tonic + " " + aard_mode, aard_r, ks_tonic + " " + ks_mode, ks_r, trevKey])
        if count % 100 == 0:
            print(f"\t{count} files processed")
            new_df = pd.DataFrame(collection, columns=['Artist', 'Song', 'Id', 'Aard_key', 'Aard_r', 'KS_key', 'KS_r', 'Trevor_key'])
            new_df.to_csv('classified_songs_V3.csv', sep='\t', index=False, mode='a')
            everything += collection
            collection = []

print(f"\t{count} files processed")
new_df = pd.DataFrame(collection, columns=['Artist', 'Song', 'Id', 'Aard_key', 'Aard_r', 'KS_key', 'KS_r', 'Trevor_key'])
new_df.to_csv('classified_songs_V3.csv', sep='\t', index=False, mode='a', header=False)
everything += collection
collection = []

folders:  ['numpy_path_0_to_200', 'numpy_path_1200_to_2400', 'numpy_path_200_to_400', 'numpy_path_2400_to_4800', 'numpy_path_400_to_600', 'numpy_path_4800_to_6000', 'numpy_path_6000_to_7200', 'numpy_path_600_to_1200', 'numpy_path_7200_to_8400', 'numpy_path_8400_to_10000']
numpy_path_0_to_200
filepath: ../numpy_data_collections/numpy_path_0_to_200/TRAANZE128F148BF55.npy
	Aarden		classification: A major	 r = 0.9020059010224748
	Krumhansl	classification: A major	 r = 0.9627136720132123
	Trevor		classification: F# minor
filepath: ../numpy_data_collections/numpy_path_0_to_200/TRADCVS128F932D857.npy
	Aarden		classification: D major	 r = 0.8764505234672945
	Krumhansl	classification: D major	 r = 0.9905044846116654
	Trevor		classification: F# minor
filepath: ../numpy_data_collections/numpy_path_0_to_200/TRAIFLV12903CCFECD.npy
	Aarden		classification: D major	 r = 0.8117698343329706
	Krumhansl	classification: A major	 r = 0.937730249889782
	Trevor		classification: E minor
filepath: ../numpy_dat

In [ ]:
new_df = pd.DataFrame(collection, columns=['Artist', 'Song', 'Id', 'Aard_key', 'Aard_r', 'KS_key', 'KS_r', 'Trevor_key'])
print(new_df)
new_df.to_csv('classified_songs_V3.csv', sep='\t', index=False)

In [101]:
test_df = pd.read_csv('./classified_songs_V3.csv', delimiter='\t')
test_df

,Artist,Song,Id,Aard_key,Aard_r,KS_key,KS_r,Trevor_key
0,Abba,The Name Of The Game,TRAANZE128F148BF55,A major,0.902006,A major,0.962714,F# minor
1,Kidz Bop Kids,Sugar Sugar,TRADCVS128F932D857,D major,0.876451,D major,0.990504,F# minor
2,Buddy Holly,Peggy Sue,TRAIFLV12903CCFECD,D major,0.811770,A major,0.937730,E minor
3,Ole Erling,Last Christmas,TRAIFVA128F426B65F,D major,0.879276,D major,0.843860,F# minor
4,Bobby,So Many Scars [extended Dynamikk Version],TRAIQCB12903CEB776,C major,0.830492,C major,0.964777,F major
